## Function for gradient of ZIP logLik

    gradPoisson <- function(parms) {
    ## count mean
    eta <- as.vector(X %*% parms[1:kx] + offsetx)
    mu <- exp(eta)
    ## binary mean
    etaz <- as.vector(Z %*% parms[(kx+1):(kx+kz)] + offsetz)
    muz <- linkinv(etaz)
  
    ## densities at 0
    clogdens0 <- -mu
    dens0 <- muz * (1 - as.numeric(Y1)) + exp(log(1 - muz) + clogdens0)

    ## working residuals  
    wres_count <- ifelse(Y1, Y - mu, -exp(-log(dens0) + log(1 - muz) + clogdens0 + log(mu)))
    wres_zero <- ifelse(Y1, -1/(1-muz) * linkobj$mu.eta(etaz),
    (linkobj$mu.eta(etaz) - exp(clogdens0) * linkobj$mu.eta(etaz))/dens0)
      
    colSums(cbind(wres_count * weights * X, wres_zero * weights * Z))
    }

In [155]:
def gradPoisson(parms):
    ## count mean
    eta = np.dot(X,parms[np.arange(kx)]) + offsetx
    mu = np.exp(eta)
    ## binary mean
    etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
    muz = linkinv(etaz)
    ## densities at 0
    clogdens0 = -mu
    dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
    ## working residuals  
    wres_count = np.where(Y1,Y-mu,-np.exp(-np.log(dens0) + 
                                          np.log(1 - muz) + clogdens0 + np.log(mu))) 
    link_etaz = np.exp(etaz)/(1+np.exp(etaz))**2
    wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
    
    
    return (np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)

In [94]:
## count mean
eta = np.dot(X,parms[np.arange(kx)]) + offsetx
mu = np.exp(eta)
## binary mean
etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
muz = linkinv(etaz)

In [144]:
(np.expand_dims(weights,axis=1)*Z).shape

(4406, 3)

In [152]:
(np.expand_dims(wres_zero*weights,axis=1)*Z).sum(axis=0)

Intercept             -55.544311
health[T.excellent]   -12.769008
health[T.poor]         -1.303869
dtype: float64

In [154]:
X.head()

,Intercept,health[T.excellent],health[T.poor],gender[T.male],privins[T.yes],hosp,numchron,school
1,1.0,0.0,0.0,1.0,1.0,1.0,2.0,6.0
2,1.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0
3,1.0,0.0,1.0,0.0,0.0,3.0,4.0,10.0
4,1.0,0.0,1.0,1.0,1.0,1.0,2.0,3.0
5,1.0,0.0,0.0,0.0,1.0,0.0,2.0,6.0


In [153]:
(np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)

array([  3.01583524e+03,   2.06987205e+02,   4.71837390e+02,
         1.31490242e+03,   2.09412944e+03,   1.04395443e+03,
         4.04704433e+03,   2.98589710e+04,  -5.55443115e+01,
        -1.27690077e+01,  -1.30386892e+00])

In [149]:
np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, np.expand_dims(wres_zero*weights,axis=1)*Z))[0]

array([-0.65859172, -0.        , -0.        , -0.65859172, -0.65859172,
       -0.65859172, -1.31718343, -3.95155029, -0.15445996, -0.        , -0.        ])

In [140]:
(wres_zero*weights).shape

(4406,)

In [141]:
(wres_zero*weights).*Z

ValueError: operands could not be broadcast together with shapes (4406,) (4406,3) 

In [135]:
print(muz[0:9])

[ 0.15445996  0.15445996  0.10649819  0.10649819  0.15445996  0.10649819
  0.15445996  0.15445996  0.15445996]


In [109]:
print(etaz)

[-1.70004062 -1.70004062 -2.12702032 ..., -1.70004062 -1.70004062
 -1.15780116]


In [125]:
np.exp(-1.7)/(1+np.exp(-1.7))**2

0.13060574696620805

In [126]:
np.exp(etaz)/(1+np.exp(etaz))**2

array([ 0.13060208,  0.13060208,  0.09515633, ...,  0.13060208,
        0.13060208,  0.181914  ])

In [15]:
(muz*(1-Y1)).shape

(4406,)

In [95]:
clogdens0 = -mu

In [18]:
(np.exp(np.log(1 - muz) + clogdens0)).shape

(4406,)

In [19]:
muz*(1-Y1) + np.exp(np.log(1 - muz) + clogdens0)

array([  1.74371236e-03,   1.28842636e-03,   9.73572911e-07, ...,
         1.72003055e-05,   4.61829167e-03,   3.72469261e-01])

In [96]:
dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)

In [24]:
(-np.exp(-np.log(dens0) + np.log(1 - muz) + clogdens0 + np.log(mu))).shape

(4406,)

In [29]:
(-1/(1-muz)*linkobj(np.exp(etaz)))

array([       -inf,        -inf,  2.23850113, ...,        -inf,
              -inf,  1.02593345])

In [97]:
    wres_count = np.where(Y1,Y-mu,-np.exp(-np.log(dens0) + 
                                          np.log(1 - muz) + clogdens0 + np.log(mu)))        


In [68]:
np.min(wres_count)

-37.35060880336647

In [76]:
Y1.astype(float)[1]

1.0

In [80]:
wres_count * np.dot(weights,X)

ValueError: operands could not be broadcast together with shapes (4406,) (4406,8) 

In [131]:
link_etaz = np.exp(etaz)/(1+np.exp(etaz))**2
wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)

In [101]:
print(wres_count)

[ -0.65859172  -4.9611865   -0.72968619 ...,  -0.27743623  11.31541677
  -0.62361333]


In [132]:
print(wres_zero)

[-0.15445996 -0.15445996 -0.10649819 ..., -0.15445996 -0.15445996
  0.40277666]


In [107]:
print(max(wres_count))

72.5959293235


In [108]:
print(max(wres_zero))

2.23850112907


In [156]:
np.max(((linkobj(np.exp(etaz)) - np.exp(clogdens0) * \
                           linkobj(np.exp(etaz)))/dens0)[~Y1])

-1.7284775958194636

In [79]:
-1/(1-muz) * linkobj(np.exp(etaz))

array([       -inf,        -inf,  2.23850113, ...,        -inf,
              -inf,  1.02593345])

In [ ]:
np.where(Y1, Y-mu, )

In [58]:
temp = np.array([True, False, True])
a = np.array([1, 2, 3])
b = np.array([8, 9 ,10])
c = np.where(temp,a,b)
print(c)

[1 9 3]


In [44]:
(-1/(1-muz) * linkobj(np.exp(etaz))).shape

(4406,)

In [46]:
((-1/(1-muz) * linkobj(np.exp(etaz)))[Y1]).shape

(3723,)

In [53]:
((-1/(1-muz) * linkobj(np.exp(etaz)))).shape

(4406,)

In [55]:
(~Y1)[1]

False

In [ ]:
wres_zero * weights * Z

In [85]:
import numpy as np
import pandas as pd
import patsy
df = pd.read_csv('DebTrivedi.csv',index_col = [0])
sel = np.array([1, 6, 7, 8, 13, 15, 18])-1
df = df.iloc[:,sel]
# produce design matrices from R-style formula
formula = 'ofp ~ hosp + health + numchron + gender + school + privins'
y, X = patsy.dmatrices(formula, df, return_type='dataframe')
# health as Z variables
Z = X.iloc[:,[0,1,2]]

In [86]:
Y = np.squeeze(y.values)
## convenience variables
n = len(Y)
kx = X.shape[1] # Number of columns in X
kz = Z.shape[1]
Y0 = Y <= 0
Y1 = Y > 0

offsetx = np.zeros(n)
offsetz = np.zeros(n)
weights = np.ones(n)

In [157]:
import statsmodels.api as sm
model_count = sm.Poisson(endog = Y, exog = X).fit()
count = model_count.params
model_zero = sm.GLM(Y0.astype(int), exog = Z, family=sm.families.Binomial()).fit()
zero = model_zero.params

Optimization terminated successfully.
         Current function value: 4.078895
         Iterations 23


In [25]:
import scipy as sp
linkinv = sp.special.expit
linkobj = sp.special.logit

In [90]:
parms = np.hstack((count.values,zero.values))

In [91]:
def ziPoisson(parms):
    ## count mean
    mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
    ## binary mean
    phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
    ## log-likelihood for y = 0 and y >= 1
    loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
    loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
    ## collect and return
    loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
    return loglik

In [92]:
ziPoisson(parms)

-16582.015992208042

In [88]:
zero

Intercept             -1.700041
health[T.excellent]    0.542239
health[T.poor]        -0.426980
dtype: float64